## Step 1: Load Libraries

In [45]:
# Install all required packages
!pip install pandas krippendorff statsmodels scikit-learn seaborn matplotlib numpy nltk


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [46]:
import pandas as pd
import krippendorff
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa
import itertools
import seaborn as sns
from scipy.stats import entropy
from sklearn.cluster import KMeans
from scipy.special import rel_entr
from scipy.stats import entropy
import itertools
from collections import Counter
import pandas as pd
import ast 
from statsmodels.stats.inter_rater import fleiss_kappa
from itertools import product
from nltk.metrics import agreement
from nltk.metrics import masi_distance
from collections import defaultdict

## Step 2: Read and Prepare Data

In [47]:
df = pd.read_csv("./gold_dataset.csv")

In [48]:
df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.text-eng,Input.label,Input.parameter,Input.parameter_num_labels,Answer.manifesto_class.labels,Approve,Reject
0,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
1,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (238/239),100% (19/19),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
2,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
3,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Keynesian Demand Management"",""Market regulat...",NaN,NaN
4,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Government and administrative efficiency"",""M...",NaN,NaN


#### **Data Cleaning and Pre-Proccessing**

Here, I want to keep HITId (represnting speeches), WorkerId, and Answer.manifesto_class.labels

In [49]:
# only keep needed columns and rename 
df_clean = df[['HITId', 'WorkerId', 'Answer.manifesto_class.labels']].copy() #remove unnecessary columns
df_clean.rename(columns={
    'HITId': 'speech_id',
    'WorkerId': 'worker_id',
    'Answer.manifesto_class.labels': 'answer'
}, inplace=True)

# extra cleaning, dropping nulls, making sure all labels are ints, 
df_clean = df_clean.sort_values(by=['speech_id', 'worker_id']).reset_index(drop=True)

# Final preview

df_clean

,speech_id,worker_id,answer
0,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1NBQ61Y6KO3O9,"[""Market regulation"",""Middle Class & Professio..."
1,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1X47COW2Y9SEL,"[""Market regulation""]"
2,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A335R4YE2E34H6,"[""Market regulation""]"
3,302OLP89EMP3TBB1087B1CXS184CAA,A1NBQ61Y6KO3O9,"[""Government and administrative efficiency"",""E..."
4,302OLP89EMP3TBB1087B1CXS184CAA,A1X47COW2Y9SEL,"[""Economic Goals""]"
...,...,...,...
655,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A1X47COW2Y9SEL,"[""Market regulation"",""Government and administr..."
656,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A3GS2NTQ4XU059,"[""Free Market Economy""]"
657,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1BQ37ZGUM16XI,"[""Market regulation""]"
658,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1X47COW2Y9SEL,"[""Market regulation"",""Anti-Growth Economy""]"


In [136]:
# sanity check

# First, ensure 'answer' is treated as a list, not a string
df_clean['answer'] = df_clean['answer'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Filter out empty annotations
df_non_empty = df_clean[df_clean['answer'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

# Count valid annotations per speech
annotation_counts = df_non_empty.groupby('speech_id').size()

# Identify speeches that don't have exactly 3 valid annotations
incomplete_speeches = annotation_counts[annotation_counts != 3]

# Print result
print(f"Speeches with <3 valid annotations: {len(incomplete_speeches)}")
print(incomplete_speeches)


Speeches with <3 valid annotations: 65
speech_id
302OLP89EMP3TBB1087B1CXS184CAA    2
309D674SIM38CLGT6M77WNHYJQPCB5    2
30IRMPJWEM1DBQ96UWAVKLZA6YPKRH    1
30OITAWPCDLG50CCUU7CG8JO8IRH99    2
3126F2F5GVLPDD1HLIV3HVMQL6HEPL    2
                                 ..
3V8JSVE8ZL8JMRTIC5RMUMF2PBKEYF    2
3VGET1QS0NHUO7J8AWVS3F25ST47W1    2
3VQTAXTYOQ34YFXTSDS7QT34QQWBUE    2
3W9XHF7WH7DQR5MYYLFQK00SUJMKTU    2
3WJGKMRWW5R1RIEDKIYDRD5YMB1CDC    2
Length: 65, dtype: int64


define 'answer's and workers

In [50]:
workers = [
    'A1BQ37ZGUM16XI', 
    'A1NBQ61Y6KO3O9', 
    'A1X47COW2Y9SEL', 
    'A335R4YE2E34H6', 
    'A3GS2NTQ4XU059', 
    'APXGSUB250NVH'
]

In [51]:
# only run this once!!!!!!!!!!!!!!!
# df_clean['answer'] = df_clean['answer'].apply(eval)  # Safely parse list from string

Rename all df columns to have a T/F for each category

In [52]:

# turn all the answers into an array 
all_answers = df_clean['answer'].dropna().apply(ast.literal_eval)
all_categories = [category for sublist in all_answers for category in sublist]
unique_categories = pd.Series(all_categories).unique()

print(unique_categories) #check 

['Market regulation' 'Middle Class & Professional Groups'
 'Government and administrative efficiency' 'Economic Planning'
 'Economic Goals' 'Anti-Growth Economy' 'Economic Growth'
 'Free Market Economy' 'Incentives' 'Keynesian Demand Management'
 'Corporatism / Mixed Economy' 'Technology and Infrastructure'
 'Labor Groups Positive' 'Labor Groups Negative']


In [53]:
for category in unique_categories:
    df[category] = df['Answer.manifesto_class.labels'].str.contains(category)

Create a secondary matrix with unique speech rows and worker columns

In [169]:
df_pivot = df_clean.pivot(index='speech_id', columns='worker_id', values='answer')
df_pivot = df_pivot.reindex(columns=workers)
df_pivot = df_pivot.fillna('[]')  # maybe change

df_pivot

worker_id,A1BQ37ZGUM16XI,A1NBQ61Y6KO3O9,A1X47COW2Y9SEL,A335R4YE2E34H6,A3GS2NTQ4XU059,APXGSUB250NVH
speech_id,,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,[],"[Market regulation, Middle Class & Professiona...",[Market regulation],[Market regulation],[],[]
302OLP89EMP3TBB1087B1CXS184CAA,[],"[Government and administrative efficiency, Eco...",[Economic Goals],[],[],[]
306996CF7J2OPI6VQPINFRY7FTYB1E,[],"[Anti-Growth Economy, Market regulation]",[Government and administrative efficiency],"[Economic Planning, Economic Growth]",[],[]
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,[],"[Market regulation, Economic Planning, Economi...","[Free Market Economy, Economic Growth]","[Market regulation, Economic Growth]",[],[]
308KJXFUKEOKFSDCMU21QMXWPU4ATR,[],"[Economic Planning, Market regulation, Economi...",[Economic Goals],"[Market regulation, Economic Planning]",[],[]
...,...,...,...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,[],"[Government and administrative efficiency, Mar...","[Government and administrative efficiency, Fre...",[],[Free Market Economy],[]
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,[],"[Corporatism / Mixed Economy, Anti-Growth Econ...",[],[Corporatism / Mixed Economy],[],"[Economic Goals, Government and administrative..."
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,[],"[Middle Class & Professional Groups, Incentive...",[Technology and Infrastructure],"[Labor Groups Positive, Technology and Infrast...",[],[]


Seperate DataFrame by categories in answers

In [126]:
df_pivot_cleaned

worker_id,A1BQ37ZGUM16XI,A1NBQ61Y6KO3O9,A1X47COW2Y9SEL,A335R4YE2E34H6,A3GS2NTQ4XU059,APXGSUB250NVH
speech_id,,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,[],"[Market regulation, Middle Class & Professiona...",[Market regulation],[Market regulation],[],[]
302OLP89EMP3TBB1087B1CXS184CAA,[],"[Government and administrative efficiency, Eco...",[Economic Goals],[],[],[]
306996CF7J2OPI6VQPINFRY7FTYB1E,[],"[Anti-Growth Economy, Market regulation]",[Government and administrative efficiency],"[Economic Planning, Economic Growth]",[],[]
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,[],"[Market regulation, Economic Planning, Economi...","[Free Market Economy, Economic Growth]","[Market regulation, Economic Growth]",[],[]
308KJXFUKEOKFSDCMU21QMXWPU4ATR,[],"[Economic Planning, Market regulation, Economi...",[Economic Goals],"[Market regulation, Economic Planning]",[],[]
...,...,...,...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,[],"[Government and administrative efficiency, Mar...","[Government and administrative efficiency, Fre...",[],[Free Market Economy],[]
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,[],"[Corporatism / Mixed Economy, Anti-Growth Econ...",[],[Corporatism / Mixed Economy],[],"[Economic Goals, Government and administrative..."
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,[],"[Middle Class & Professional Groups, Incentive...",[Technology and Infrastructure],"[Labor Groups Positive, Technology and Infrast...",[],[]


In [ ]:
# # Make sure cell values are lists (not strings that look like lists)
# df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)

# # This will store each binary dataframe
# category_dfs = {}

# # For each unique category
# for category in unique_categories:

#    # HERE change df_binary to have rows be speeches in category, and columns be arbitrary workers: w1,w2,w3
#     # df_binary.insert('w1','w2','w3')
    

#     # HERE change the mapping here so that if (sppech, workerid) is empty string[] in df_pivoted then skip in df_binary !!! else: in cell --> 1, if not --> 0 
#     df_binary = df_pivot_cleaned.applymap(lambda x: 1 if category in x else 0 if category not in x else None)
    
#     # Store it
#     category_dfs[category] = df_binary

# category_dfs['Market regulation'] # test one category


/var/folders/m1/tw6x4xns6fn4nk3wf2lc_3vc0000gn/T/ipykernel_63980/2541683071.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)
/var/folders/m1/tw6x4xns6fn4nk3wf2lc_3vc0000gn/T/ipykernel_63980/2541683071.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_binary = df_pivot_cleaned.applymap(lambda x: 1 if category in x else 0 if category not in x else None)


worker_id,A1BQ37ZGUM16XI,A1NBQ61Y6KO3O9,A1X47COW2Y9SEL,A335R4YE2E34H6,A3GS2NTQ4XU059,APXGSUB250NVH
speech_id,,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,0,1,1,1,0,0
302OLP89EMP3TBB1087B1CXS184CAA,0,0,0,0,0,0
306996CF7J2OPI6VQPINFRY7FTYB1E,0,1,0,0,0,0
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,0,1,0,1,0,0
308KJXFUKEOKFSDCMU21QMXWPU4ATR,0,1,0,1,0,0
...,...,...,...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,0,1,0,0,0,0
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,0,0,0,0,0,0
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,0,0,0,0,0,0


In [141]:
# Step 1: Ensure each cell is a list (not a string)
df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)

# Step 2: Create binary DataFrames per category
category_dfs = {}

for category in unique_categories:
    rows = []
    speech_ids = []

    for speech_id, row in df_pivot_cleaned.iterrows():
        votes = []

        for annotations in row.values:
            if isinstance(annotations, list) and annotations:  # non-empty list
                vote = 1 if category in annotations else 0
                votes.append(vote)

        # Only include rows with exactly 3 annotations
        if len(votes) == 3:
            rows.append(votes)
            speech_ids.append(speech_id)

    df_binary = pd.DataFrame(rows, index=speech_ids, columns=['worker1', 'worker2', 'worker3'])
    category_dfs[category] = df_binary

category_dfs['Market regulation'] # test one category

/var/folders/m1/tw6x4xns6fn4nk3wf2lc_3vc0000gn/T/ipykernel_63980/2877661995.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)


,worker1,worker2,worker3
301KG0KXAZ355GRD9LSX6LBA2CHH2H,1,1,1
306996CF7J2OPI6VQPINFRY7FTYB1E,1,0,0
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,1,0,1
308KJXFUKEOKFSDCMU21QMXWPU4ATR,1,0,1
30EMX9PEW71PC59J9LA0AV46S08KS5,1,0,0
...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,1,0,0
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,0,0,0
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,0,0,0
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,1,1,0


## Try 1 : Use a massive aggregate Dataframe (didn't work)

In [55]:

# turn all the answers into an array 
all_answers = df_clean['answer'].dropna().apply(ast.literal_eval)
all_categories = [category for sublist in all_answers for category in sublist]
unique_categories = pd.Series(all_categories).unique()

print(unique_categories) #check 


['Market regulation' 'Middle Class & Professional Groups'
 'Government and administrative efficiency' 'Economic Planning'
 'Economic Goals' 'Anti-Growth Economy' 'Economic Growth'
 'Free Market Economy' 'Incentives' 'Keynesian Demand Management'
 'Corporatism / Mixed Economy' 'Technology and Infrastructure'
 'Labor Groups Positive' 'Labor Groups Negative']


Now, we compute a 'count' matrix in order to analyze similary with fleiss kappa; here columns are (answer, annotator) pairs

In [56]:
# Clean and parse the 'answer' column into lists
df_clean['parsed_answer'] = df_clean['answer'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) and x.strip() else []
)

# Get unique workers and categories
unique_workers = df_clean['worker_id'].unique()
all_categories = [cat for ans in df_clean['parsed_answer'] for cat in ans]
unique_categories = pd.Series(all_categories).unique()


print(unique_categories) #view


['Market regulation' 'Middle Class & Professional Groups'
 'Government and administrative efficiency' 'Economic Planning'
 'Economic Goals' 'Anti-Growth Economy' 'Economic Growth'
 'Free Market Economy' 'Incentives' 'Keynesian Demand Management'
 'Corporatism / Mixed Economy' 'Technology and Infrastructure'
 'Labor Groups Positive' 'Labor Groups Negative']


In [57]:
# construct binary matrix 
worker_category_pairs = list(product(unique_workers, unique_categories))
speech_ids = df_clean['speech_id'].unique()
binary_matrix = pd.DataFrame(0, index=speech_ids, columns=worker_category_pairs)

print(binary_matrix.columns) #view



Index([                       ('A1NBQ61Y6KO3O9', 'Market regulation'),
             ('A1NBQ61Y6KO3O9', 'Middle Class & Professional Groups'),
       ('A1NBQ61Y6KO3O9', 'Government and administrative efficiency'),
                              ('A1NBQ61Y6KO3O9', 'Economic Planning'),
                                 ('A1NBQ61Y6KO3O9', 'Economic Goals'),
                            ('A1NBQ61Y6KO3O9', 'Anti-Growth Economy'),
                                ('A1NBQ61Y6KO3O9', 'Economic Growth'),
                            ('A1NBQ61Y6KO3O9', 'Free Market Economy'),
                                     ('A1NBQ61Y6KO3O9', 'Incentives'),
                    ('A1NBQ61Y6KO3O9', 'Keynesian Demand Management'),
                    ('A1NBQ61Y6KO3O9', 'Corporatism / Mixed Economy'),
                  ('A1NBQ61Y6KO3O9', 'Technology and Infrastructure'),
                          ('A1NBQ61Y6KO3O9', 'Labor Groups Positive'),
                          ('A1NBQ61Y6KO3O9', 'Labor Groups Negative'),
      

In [58]:


# loop through speeches in binary_matrix - speech_id are the index and don't have a column
    # loop through workers - stored in workers[] array
        # if this worker mentioned its paired answer in this speech, assign this cell to 1
# Ensure columns are MultiIndex
if not isinstance(binary_matrix.columns, pd.MultiIndex):
    binary_matrix.columns = pd.MultiIndex.from_tuples(binary_matrix.columns)

# Get the list of speech IDs and workers
speech_ids = binary_matrix.index
workers = list(set(df_clean['worker_id']))

# Loop through each speech
for speech_id in speech_ids:
    # Filter df_clean to just this speech
    df_speech = df_clean[df_clean['speech_id'] == speech_id]

    # Loop through each worker
    for worker in workers:
        # Get the categories this worker annotated for this speech
        categories = df_speech[df_speech['worker_id'] == worker]['parsed_answer']
        if categories.empty:
            continue  # this worker didn't annotate this speech

        # categories is a Series of lists → flatten it
        mentioned_cats = set(cat for sublist in categories for cat in sublist)

        for category in mentioned_cats:
            key = (worker, category)
            if key in binary_matrix.columns:
                binary_matrix.loc[speech_id, key] = 1




binary_matrix #view 




A1NBQ61Y6KO3O9  \
                               Market regulation   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 1   
302OLP89EMP3TBB1087B1CXS184CAA                 0   
306996CF7J2OPI6VQPINFRY7FTYB1E                 1   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1   
...                                          ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0   

                                                                   \
                               Middle Class & Professional Groups   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                  1   
302OLP89EMP3TBB1087B1CXS184CAA                                  0   
306996CF7J2OPI6VQPINFRY7FTYB1E                                  0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                  0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                  0   
...                                                           ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                  0   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                  0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                  1   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                  1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                  0   

                                                                         \
                               Government and administrative efficiency   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                        0   
302OLP89EMP3TBB1087B1CXS184CAA                                        1   
306996CF7J2OPI6VQPINFRY7FTYB1E                                        0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                        0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                        0   
...                                                                 ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                        1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                        1   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                        0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                        1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                        0   

                                                                 \
                               Economic Planning Economic Goals   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 0              0   
302OLP89EMP3TBB1087B1CXS184CAA                 1              1   
306996CF7J2OPI6VQPINFRY7FTYB1E                 0              0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1              0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1              1   
...                                          ...            ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 0              1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0              0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0              0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 0              0   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0              0   

                                                                    \
                               Anti-Growth Economy Economic Growth   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                   0               0   
302OLP89EMP3TBB1087B1CXS184CAA                   0               0   
306996CF7J2OPI6VQPINFRY7FTYB1E                   1               0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                   0               1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                   0               0   
...                                            ...             ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                   0 

Check binary_matrix columns to ensure data aggregation worked 

In [59]:

binary_matrix

A1NBQ61Y6KO3O9  \
                               Market regulation   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 1   
302OLP89EMP3TBB1087B1CXS184CAA                 0   
306996CF7J2OPI6VQPINFRY7FTYB1E                 1   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1   
...                                          ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0   

                                                                   \
                               Middle Class & Professional Groups   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                  1   
302OLP89EMP3TBB1087B1CXS184CAA                                  0   
306996CF7J2OPI6VQPINFRY7FTYB1E                                  0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                  0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                  0   
...                                                           ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                  0   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                  0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                  1   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                  1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                  0   

                                                                         \
                               Government and administrative efficiency   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                        0   
302OLP89EMP3TBB1087B1CXS184CAA                                        1   
306996CF7J2OPI6VQPINFRY7FTYB1E                                        0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                        0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                        0   
...                                                                 ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                        1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                        1   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                        0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                        1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                        0   

                                                                 \
                               Economic Planning Economic Goals   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 0              0   
302OLP89EMP3TBB1087B1CXS184CAA                 1              1   
306996CF7J2OPI6VQPINFRY7FTYB1E                 0              0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1              0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1              1   
...                                          ...            ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 0              1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0              0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0              0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 0              0   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0              0   

                                                                    \
                               Anti-Growth Economy Economic Growth   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                   0               0   
302OLP89EMP3TBB1087B1CXS184CAA                   0               0   
306996CF7J2OPI6VQPINFRY7FTYB1E                   1               0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                   0               1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                   0               0   
...                                            ...             ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                   0 

In [60]:
# kappa = fleiss_kappa(binary_matrix)
# print(f"{category}: Fleiss’ Kappa = {kappa:.3f}")
    

## Try 2: run Fleiss Kappa on each seperate category

In [147]:
for category, category_dfs[category] in category_dfs.items():
    try:
        # For each row (speech), count number of annotators giving 0 and 1
        counts_matrix = []

        for _, row in category_dfs[category].iterrows():
            counts = [list(row).count(0), list(row).count(1)]  # [num 0s, num 1s]
            counts_matrix.append(counts)

        counts_array = np.array(counts_matrix)
        
        # Compute Fleiss' Kappa
        kappa = fleiss_kappa(counts_array)
        print(f"{category}: Fleiss’ Kappa = {kappa:.3f}")

    except Exception as e:
        print(f"Error computing Fleiss’ Kappa for {category}: {e}")

Market regulation: Fleiss’ Kappa = 0.289
Middle Class & Professional Groups: Fleiss’ Kappa = 0.001
Government and administrative efficiency: Fleiss’ Kappa = 0.179
Economic Planning: Fleiss’ Kappa = 0.173
Economic Goals: Fleiss’ Kappa = 0.065
Anti-Growth Economy: Fleiss’ Kappa = 0.092
Economic Growth: Fleiss’ Kappa = 0.460
Free Market Economy: Fleiss’ Kappa = 0.280
Incentives: Fleiss’ Kappa = 0.236
Keynesian Demand Management: Fleiss’ Kappa = 0.203
Corporatism / Mixed Economy: Fleiss’ Kappa = 0.058
Technology and Infrastructure: Fleiss’ Kappa = 0.409
Labor Groups Positive: Fleiss’ Kappa = 0.305
Labor Groups Negative: Fleiss’ Kappa = -0.002


This is a bad number likely due to sparse annotations

In [143]:
[(cat, df.sum().sum()) for cat, df in category_dfs.items()]

[('Market regulation', np.int64(196)),
 ('Middle Class & Professional Groups', np.int64(36)),
 ('Government and administrative efficiency', np.int64(106)),
 ('Economic Planning', np.int64(96)),
 ('Economic Goals', np.int64(70)),
 ('Anti-Growth Economy', np.int64(36)),
 ('Economic Growth', np.int64(68)),
 ('Free Market Economy', np.int64(38)),
 ('Incentives', np.int64(74)),
 ('Keynesian Demand Management', np.int64(34)),
 ('Corporatism / Mixed Economy', np.int64(12)),
 ('Technology and Infrastructure', np.int64(29)),
 ('Labor Groups Positive', np.int64(64)),
 ('Labor Groups Negative', np.int64(1))]

## Try 3: run Krippendorff’s Alpha on annotations

 Krippendorff’s Alpha doesn't require that each annotator label once, so this measure will work for us. We can either run this measure on the entire dataset, or seperate by category and run on each one. 

In [63]:
# first attempt to run krippendorff on the entire dataset
alpha_value = krippendorff.alpha(
    reliability_data=binary_matrix,
    level_of_measurement='nominal'  # Change to 'interval', 'ordinal', etc. if needed
)

print(f"Krippendorff’s Alpha: {alpha_value:.3f}")




Krippendorff’s Alpha: 0.116


This number is not extremely good; therefore, I will run Krippendorffs on every category. First, I need to make new dataframes for each category. 

In [ ]:
#check category_dfs to see if its working
category_dfs['Market regulation'].head()

worker_id,A1BQ37ZGUM16XI,A1NBQ61Y6KO3O9,A1X47COW2Y9SEL,A335R4YE2E34H6,A3GS2NTQ4XU059,APXGSUB250NVH
speech_id,,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,0,1,1,1,0,0
302OLP89EMP3TBB1087B1CXS184CAA,0,0,0,0,0,0
306996CF7J2OPI6VQPINFRY7FTYB1E,0,1,0,0,0,0
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,0,1,0,1,0,0
308KJXFUKEOKFSDCMU21QMXWPU4ATR,0,1,0,1,0,0


Now we calculate each score. It is important to note we are measuring agreement on whether a single label was applied, not overall multi-label set overlap.

In [92]:
for category, df_binary in category_dfs.items():
    try:
        # Convert to a list of lists (rows = items, columns = annotators)
        data_matrix = category_dfs[category].to_numpy().tolist()
        
        # Compute Krippendorff's alpha
        alpha_score = krippendorff.alpha(
            reliability_data=data_matrix,
            level_of_measurement='nominal'
        )
        
        print(f"Category: {category} → Krippendorff’s alpha: {alpha_score:.3f}")
    
    except Exception as e:
        print(f"Error for category {category}: {e}")

Category: Market regulation → Krippendorff’s alpha: 0.133
Category: Middle Class & Professional Groups → Krippendorff’s alpha: 0.105
Category: Government and administrative efficiency → Krippendorff’s alpha: 0.175
Category: Economic Planning → Krippendorff’s alpha: 0.047
Category: Economic Goals → Krippendorff’s alpha: 0.041
Category: Anti-Growth Economy → Krippendorff’s alpha: 0.069
Category: Economic Growth → Krippendorff’s alpha: 0.049
Category: Free Market Economy → Krippendorff’s alpha: 0.011
Category: Incentives → Krippendorff’s alpha: 0.101
Category: Keynesian Demand Management → Krippendorff’s alpha: 0.007
Category: Corporatism / Mixed Economy → Krippendorff’s alpha: 0.006
Category: Technology and Infrastructure → Krippendorff’s alpha: 0.011
Category: Labor Groups Positive → Krippendorff’s alpha: 0.046
Category: Labor Groups Negative → Krippendorff’s alpha: 0.000


Interpretation: very low agreement, try to analyze more

In [80]:
df = category_dfs['Market regulation']
total_cells = df.shape[0] * df.shape[1]
positive = df.sum().sum()
sparsity = positive / total_cells

print(f"Total annotations: {total_cells}")
print(f"Positive annotations: {positive}")
print(f"Sparsity (positive rate): {sparsity:.2%}")

Total annotations: 1320
Positive annotations: 238
Sparsity (positive rate): 18.03%


## Try 4: run pair wise Jaccard Similarity per Speech (ignore)

In [81]:
df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)

def jaccard(set1, set2):
    union = set1 | set2
    return len(set1 & set2) / len(union) if union else 1.0  # define J(∅, ∅) = 1.0

# Store average Jaccard per speech
jaccard_scores = {}

for speech_id, row in df_pivot_cleaned.iterrows():
    # Convert each cell to a set of labels
    label_sets = [set(val) for val in row.dropna() if isinstance(val, list)]
    
    # Skip speeches with < 2 annotators
    if len(label_sets) < 2:
        continue

    # All pairwise Jaccard similarities
    pairwise_sims = [
        jaccard(a, b) for a, b in itertools.combinations(label_sets, 2)
    ]

    avg_sim = sum(pairwise_sims) / len(pairwise_sims)
    jaccard_scores[speech_id] = avg_sim

# Convert to DataFrame
df_jaccard = pd.DataFrame.from_dict(jaccard_scores, orient='index', columns=['avg_jaccard_similarity'])
df_jaccard.index.name = 'speech_id'
print(df_jaccard.describe())

       avg_jaccard_similarity
count              220.000000
mean                 0.366229
std                  0.186314
min                  0.200000
25%                  0.238056
50%                  0.292063
75%                  0.422222
max                  1.000000


/var/folders/m1/tw6x4xns6fn4nk3wf2lc_3vc0000gn/T/ipykernel_63980/2763074208.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)


In [82]:
df_jaccard.sort_values('avg_jaccard_similarity', ascending=False)

,avg_jaccard_similarity
speech_id,
3MVY4USGCT5Y6AJV9IPOXFPT0WJISS,1.0
3MZ3TAMYU85M5VJIUTPJ75KXEU0IR4,1.0
3R868ACW5FUHBVZEEMJT8W9G8L3GZP,1.0
3WUVMVA7PYLG14MBSMRJCAYDAQQAZ7,1.0
39N6W9XWS05H65P9I0HAUP0E2RZGYY,1.0
...,...
3SBNLSTU7HNEZBPNCWIL2B6JV6BDZQ,0.2
306996CF7J2OPI6VQPINFRY7FTYB1E,0.2
3UXQ63NLBX41FPAZJZ06Z67LDCRLB8,0.2


In [70]:
print(f"Mean Jaccard similarity across all speeches: {df_jaccard['avg_jaccard_similarity'].mean():.3f}")


Mean Jaccard similarity across all speeches: 0.366


In [71]:
worker_sim_scores = defaultdict(list)  # {worker_id: [similarity scores]}

# Iterate over each speech
for _, row in df_pivot_cleaned.iterrows():
    worker_sets = {
        worker: set(val) for worker, val in row.items()
        if isinstance(val, list)  # only valid annotations
    }

    workers = list(worker_sets.keys())
    for w1, w2 in itertools.combinations(workers, 2):
        sim = jaccard(worker_sets[w1], worker_sets[w2])
        worker_sim_scores[w1].append(sim)
        worker_sim_scores[w2].append(sim)  # symmetric

# Now compute average consistency per worker
worker_consistency = {
    worker: sum(scores)/len(scores) if scores else np.nan
    for worker, scores in worker_sim_scores.items()
}

# Convert to DataFrame
df_worker_consistency = pd.DataFrame.from_dict(worker_consistency, orient='index', columns=['avg_worker_similarity'])
df_worker_consistency.index.name = 'worker_id'

print(df_worker_consistency.sort_values('avg_worker_similarity', ascending=False))

                avg_worker_similarity
worker_id                            
A1BQ37ZGUM16XI               0.493485
APXGSUB250NVH                0.429303
A3GS2NTQ4XU059               0.367667
A1X47COW2Y9SEL               0.351745
A335R4YE2E34H6               0.347844
A1NBQ61Y6KO3O9               0.207329


## Cutting Workers Pt. 1 (Ignore)

First, we take out potential outliers,I hypothesize workers with little entries may bring down the fleiss kappa. 

In [ ]:
df_clean['worker_id'].value_counts()

worker_id
A1NBQ61Y6KO3O9    186
A335R4YE2E34H6    152
A1X47COW2Y9SEL    128
A3GS2NTQ4XU059    104
APXGSUB250NVH      83
A1BQ37ZGUM16XI      7
Name: count, dtype: int64

Now, we cut worker A1BQ37ZGUM16XI

In [124]:
# for each category_df[category], we must cut worker A1BQ37ZGUM16XI
worker_to_remove = 'A3GS2NTQ4XU059'

# Loop through each category and drop the worker
for category, df in category_dfs.items():
    if worker_to_remove in df.columns:
        category_dfs[category] = df.drop(columns=[worker_to_remove])
category_dfs['Market regulation']

,worker1,worker2,worker3
301KG0KXAZ355GRD9LSX6LBA2CHH2H,1,1,1
306996CF7J2OPI6VQPINFRY7FTYB1E,1,0,0
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,1,0,1
308KJXFUKEOKFSDCMU21QMXWPU4ATR,1,0,1
30EMX9PEW71PC59J9LA0AV46S08KS5,1,0,0
...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,1,0,0
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,0,0,0
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,0,0,0
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,1,1,0


In [125]:

for category, category_dfs[category] in category_dfs.items():
    try:
        # For each row (speech), count number of annotators giving 0 and 1
        counts_matrix = []

        for _, row in category_dfs[category].iterrows():
            counts = [list(row).count(0), list(row).count(1)]  # [num 0s, num 1s]
            counts_matrix.append(counts)

        counts_array = np.array(counts_matrix)
        
        # Compute Fleiss' Kappa
        kappa = fleiss_kappa(counts_array)
        print(f"{category}: Fleiss’ Kappa = {kappa:.3f}")

    except Exception as e:
        print(f"Error computing Fleiss’ Kappa for {category}: {e}")

Market regulation: Fleiss’ Kappa = 0.146
Middle Class & Professional Groups: Fleiss’ Kappa = -0.025
Government and administrative efficiency: Fleiss’ Kappa = 0.034
Economic Planning: Fleiss’ Kappa = 0.146
Economic Goals: Fleiss’ Kappa = 0.044
Anti-Growth Economy: Fleiss’ Kappa = 0.084
Economic Growth: Fleiss’ Kappa = 0.410
Free Market Economy: Fleiss’ Kappa = 0.148
Incentives: Fleiss’ Kappa = 0.244
Keynesian Demand Management: Fleiss’ Kappa = 0.034
Corporatism / Mixed Economy: Fleiss’ Kappa = 0.058
Technology and Infrastructure: Fleiss’ Kappa = 0.387
Labor Groups Positive: Fleiss’ Kappa = 0.230
Labor Groups Negative: Fleiss’ Kappa = -0.002


## Interpretations

**Original Fliess Kappa with nulls as 0s**

<p>Market regulation: Fleiss’ Kappa = 0.051<br>
Middle Class & Professional Groups: Fleiss’ Kappa = -0.003<br>
Government and administrative efficiency: Fleiss’ Kappa = 0.024<br>
Economic Planning: Fleiss’ Kappa = 0.053<br>
Economic Goals: Fleiss’ Kappa = 0.013<br>
Anti-Growth Economy: Fleiss’ Kappa = 0.023<br>
Economic Growth: Fleiss’ Kappa = 0.155<br>
Free Market Economy: Fleiss’ Kappa = 0.087<br>
Incentives: Fleiss’ Kappa = 0.079<br>
Keynesian Demand Management: Fleiss’ Kappa = 0.078<br>
Corporatism / Mixed Economy: Fleiss’ Kappa = 0.018<br>
Technology and Infrastructure: Fleiss’ Kappa = 0.131<br>
Labor Groups Positive: Fleiss’ Kappa = 0.104<br>
Labor Groups Negative: Fleiss’ Kappa = -0.001</p>

**Fliess Kappa with arbitrary workers**
*These use less speeches, since 65 speeches had only two answer or less*

<p>Market regulation: Fleiss’ Kappa = 0.289<br>
Middle Class & Professional Groups: Fleiss’ Kappa = 0.001<br>
Government and administrative efficiency: Fleiss’ Kappa = 0.179<br>
Economic Planning: Fleiss’ Kappa = 0.173<br>
Economic Goals: Fleiss’ Kappa = 0.065<br>
Anti-Growth Economy: Fleiss’ Kappa = 0.092<br>
Economic Growth: Fleiss’ Kappa = 0.460<br>
Free Market Economy: Fleiss’ Kappa = 0.280<br>
Incentives: Fleiss’ Kappa = 0.236<br>
Keynesian Demand Management: Fleiss’ Kappa = 0.203<br>
Corporatism / Mixed Economy: Fleiss’ Kappa = 0.058<br>
Technology and Infrastructure: Fleiss’ Kappa = 0.409<br>
Labor Groups Positive: Fleiss’ Kappa = 0.305<br>
Labor Groups Negative: Fleiss’ Kappa = -0.002</p>

## Cutting Workers Pt. 2

First, we will cut workers from df_clean

In [148]:
df_clean['worker_id'].value_counts()

worker_id
A1NBQ61Y6KO3O9    186
A335R4YE2E34H6    152
A1X47COW2Y9SEL    128
A3GS2NTQ4XU059    104
APXGSUB250NVH      83
A1BQ37ZGUM16XI      7
Name: count, dtype: int64

In [ ]:
# for each category_df[category], we must cut worker A1BQ37ZGUM16XI
workers_to_remove = ['A1BQ37ZGUM16XI']

# drop this column from df_pivot
df_pivot.drop(columns = workers_to_remove, inplace=True)
df_pivot

worker_id,A1BQ37ZGUM16XI,A1NBQ61Y6KO3O9,A1X47COW2Y9SEL,A335R4YE2E34H6,A3GS2NTQ4XU059
speech_id,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,[],"[Market regulation, Middle Class & Professiona...",[Market regulation],[Market regulation],[]
302OLP89EMP3TBB1087B1CXS184CAA,[],"[Government and administrative efficiency, Eco...",[Economic Goals],[],[]
306996CF7J2OPI6VQPINFRY7FTYB1E,[],"[Anti-Growth Economy, Market regulation]",[Government and administrative efficiency],"[Economic Planning, Economic Growth]",[]
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,[],"[Market regulation, Economic Planning, Economi...","[Free Market Economy, Economic Growth]","[Market regulation, Economic Growth]",[]
308KJXFUKEOKFSDCMU21QMXWPU4ATR,[],"[Economic Planning, Market regulation, Economi...",[Economic Goals],"[Market regulation, Economic Planning]",[]
...,...,...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,[],"[Government and administrative efficiency, Mar...","[Government and administrative efficiency, Fre...",[],[Free Market Economy]
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,[],"[Corporatism / Mixed Economy, Anti-Growth Econ...",[],[Corporatism / Mixed Economy],[]
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,[],"[Middle Class & Professional Groups, Incentive...",[Technology and Infrastructure],"[Labor Groups Positive, Technology and Infrast...",[]


Then, we will re-run the script to pivot the dataframe and recompute the dictionary of different categorical dataframes

In [171]:
# Step 1: Ensure each cell is a list (not a string)
df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)

# Step 2: Create binary DataFrames per category
category_dfs = {}

for category in unique_categories:
    rows = []
    speech_ids = []

    for speech_id, row in df_pivot_cleaned.iterrows():
        votes = []

        for annotations in row.values:
            if isinstance(annotations, list) and annotations:  # non-empty list
                vote = 1 if category in annotations else 0
                votes.append(vote)

        # Only include rows with exactly 3 annotations
        if len(votes) == 3:
            rows.append(votes)
            speech_ids.append(speech_id)

    df_binary = pd.DataFrame(rows, index=speech_ids, columns=['worker1', 'worker2', 'worker3'])
    category_dfs[category] = df_binary

category_dfs['Market regulation'] # test one category

/var/folders/m1/tw6x4xns6fn4nk3wf2lc_3vc0000gn/T/ipykernel_63980/2877661995.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pivot_cleaned = df_pivot.applymap(lambda x: eval(x) if isinstance(x, str) else x)


,worker1,worker2,worker3
301KG0KXAZ355GRD9LSX6LBA2CHH2H,1,1,1
306996CF7J2OPI6VQPINFRY7FTYB1E,1,0,0
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,1,0,1
308KJXFUKEOKFSDCMU21QMXWPU4ATR,1,0,1
30EMX9PEW71PC59J9LA0AV46S08KS5,1,0,0
...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,1,0,0
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,0,0,0
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,0,0,0
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,1,1,0


Finally, we will re-compute the Fleiss Kappa scores and look at any differences

In [172]:
for category, category_dfs[category] in category_dfs.items():
    try:
        # For each row (speech), count number of annotators giving 0 and 1
        counts_matrix = []

        for _, row in category_dfs[category].iterrows():
            counts = [list(row).count(0), list(row).count(1)]  # [num 0s, num 1s]
            counts_matrix.append(counts)

        counts_array = np.array(counts_matrix)
        
        # Compute Fleiss' Kappa
        kappa = fleiss_kappa(counts_array)
        print(f"{category}: Fleiss’ Kappa = {kappa:.3f}")

    except Exception as e:
        print(f"Error computing Fleiss’ Kappa for {category}: {e}")

Market regulation: Fleiss’ Kappa = 0.289
Middle Class & Professional Groups: Fleiss’ Kappa = 0.001
Government and administrative efficiency: Fleiss’ Kappa = 0.179
Economic Planning: Fleiss’ Kappa = 0.173
Economic Goals: Fleiss’ Kappa = 0.065
Anti-Growth Economy: Fleiss’ Kappa = 0.092
Economic Growth: Fleiss’ Kappa = 0.460
Free Market Economy: Fleiss’ Kappa = 0.280
Incentives: Fleiss’ Kappa = 0.236
Keynesian Demand Management: Fleiss’ Kappa = 0.203
Corporatism / Mixed Economy: Fleiss’ Kappa = 0.058
Technology and Infrastructure: Fleiss’ Kappa = 0.409
Labor Groups Positive: Fleiss’ Kappa = 0.305
Labor Groups Negative: Fleiss’ Kappa = -0.002


**Old with last worker (for comparisons):**

<p>Market regulation: Fleiss’ Kappa = 0.289<br>
Middle Class & Professional Groups: Fleiss’ Kappa = 0.001<br>
Government and administrative efficiency: Fleiss’ Kappa = 0.179<br>
Economic Planning: Fleiss’ Kappa = 0.173<br>
Economic Goals: Fleiss’ Kappa = 0.065<br>
Anti-Growth Economy: Fleiss’ Kappa = 0.092<br>
Economic Growth: Fleiss’ Kappa = 0.460<br>
Free Market Economy: Fleiss’ Kappa = 0.280<br>
Incentives: Fleiss’ Kappa = 0.236<br>
Keynesian Demand Management: Fleiss’ Kappa = 0.203<br>
Corporatism / Mixed Economy: Fleiss’ Kappa = 0.058<br>
Technology and Infrastructure: Fleiss’ Kappa = 0.409<br>
Labor Groups Positive: Fleiss’ Kappa = 0.305<br>
Labor Groups Negative: Fleiss’ Kappa = -0.002</p>